In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

In [ ]:
llm = GoogleGenerativeAI(model="gemini-pro")

In [ ]:
from langchain import PromptTemplate
from langchain.llms import GooglePalm
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [ ]:
BASELINE_PROMPT = """Answer the below question which is asking for a concise factual answer. NO ADDITIONAL DETAILS.

Question: {query}

Answer:"""


# Chain to generate initial response
baseline_response_prompt_template = PromptTemplate.from_template(BASELINE_PROMPT)
baseline_response_chain = baseline_response_prompt_template | llm | StrOutputParser()

In [ ]:
VERIFICATION_QUESTION_TEMPLATE = """Your task is to create a verification question based on the below question provided.
Example Question: Who wrote the book 'God of Small Things' ?
Example Verification Question: Was book [God of Small Things] written by [writer]? If not who wrote [God of Small Things] ? 
Explanation: In the above example the verification question focused only on the ANSWER_ENTITY (name of the writer) and QUESTION_ENTITY (book name).
Similarly you need to focus on the ANSWER_ENTITY and QUESTION_ENTITY from the actual question and generate verification question.

Actual Question: {query}

Final Verification Question:"""


# Chain to generate a question template for verification answers
verification_question_template_prompt_template = PromptTemplate.from_template(VERIFICATION_QUESTION_TEMPLATE)
verification_question_template_chain = verification_question_template_prompt_template | llm | StrOutputParser()

In [ ]:
VERIFICATION_QUESTION_PROMPT= """Your task is to create a series of verification questions based on the below question, the verfication question template and baseline response.
Example Question: Who wrote the book 'God of Small Things' ?
Example Verification Question Template: Was book [God of Small Things] written by [writer]? If not who wrote [God of Small Things]?
Example Baseline Response: Jhumpa Lahiri
Example Verification Question: 1. Was God of Small Things written by Jhumpa Lahiri? If not who wrote God of Small Things ?


Explanation: In the above example the verification questions focused only on the ANSWER_ENTITY (name of the writer) and QUESTION_ENTITY (name of book) based on the template and substitutes entity values from the baseline response.
Similarly you need to focus on the ANSWER_ENTITY and QUESTION_ENTITY from the actual question and substitute the entity values from the baseline response to generate verification questions.

Actual Question: {query}
Baseline Response: {base_response}
Verification Question Template: {verification_question_template}

Final Verification Questions:"""


# Chain to generate the verification questions
verification_question_generation_prompt_template = PromptTemplate.from_template(VERIFICATION_QUESTION_PROMPT)
verification_question_generation_chain = verification_question_generation_prompt_template | llm | StrOutputParser()

In [ ]:
from langchain.agents import ConversationalChatAgent, AgentExecutor
from langchain.tools import DuckDuckGoSearchResults

#create search agent
search = DuckDuckGoSearchResults()
tools = [search]
custom_system_message = "Assistant assumes no knowledge & relies on internet search to answer user's queries."
max_agent_iterations = 5
max_execution_time = 10

chat_agent = ConversationalChatAgent.from_llm_and_tools(
    llm=llm, tools=tools, system_message=custom_system_message
)
search_executor = AgentExecutor.from_agent_and_tools(
    agent=chat_agent,
    tools=tools,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=max_agent_iterations,
    max_execution_time = max_execution_time
)

# chain to execute verification questions
verification_chain = RunnablePassthrough.assign(
    split_questions=lambda x: x['verification_questions'].split("\n"), # each verification question is passed one by one factored approach
) | RunnablePassthrough.assign(
    answers = (lambda x: [{"input": q,"chat_history": []} for q in x['split_questions']])| search_executor.map() # search executed for each question independently
) | (lambda x: "\n".join(["Question: {} Answer: {}\n".format(question, answer['output']) for question, answer in zip(x['split_questions'], x['answers'])]))# Create final refined response

In [ ]:
FINAL_ANSWER_PROMPT= """Given the below `Original Query` and `Baseline Answer`, analyze the `Verification Questions & Answers` to finally provide the refined answer.
Original Query: {query}
Baseline Answer: {base_response}

Verification Questions & Answer Pairs:
{verification_answers}

Final Refined Answer:"""


# Chain to generate the final answer
final_answer_prompt_template = PromptTemplate.from_template(FINAL_ANSWER_PROMPT)
final_answer_chain = final_answer_prompt_template | llm | StrOutputParser()

In [ ]:
chain = RunnablePassthrough.assign(
    base_response=baseline_response_chain
) |  RunnablePassthrough.assign(
    verification_question_template=verification_question_template_chain
) | RunnablePassthrough.assign(
    verification_questions=verification_question_generation_chain
) | RunnablePassthrough.assign(
    verification_answers=verification_chain
) | RunnablePassthrough.assign(
    final_answer=final_answer_chain
)

response = chain.invoke({"query": "Who wrote the book 'Economics of Small Things' ?"})

In [ ]:
from pprint import pprint 
pprint(response)

In [ ]:
print(response['final_answer'])

In [ ]:
response = chain.invoke({"query": "Who is 'Charan H U', does he creates  youtube videos in kannada?, which is his latest video"})
print(response['final_answer'])

In [ ]:
pprint(response)

In [10]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchRun()
search.run("Charan H U?")

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


'Hello friends! Welcome to Charan H U YouTube channel. In this video I explained about Desk Setup Tour 2023! | ಡೆಸ್ಕ್ ಸೆಟಪ್ ಪ್ರವಾಸ 2023! | Charan H Uಚರಣ್ ... 1 Why explainability and transparency matter 2 What are the challenges of AI explainability and transparency 3 What are some strategies for AI explainability and transparency 4 What are some... "We update our memories through the act of remembering," says Charan Ranganath, a professor of psychology and neuroscience at the University of California, Davis, and the author of the... DS.dev is an initiative taken by Dan and me to make Data structures and Algorithms accessible to all. We wrote the project in React, working on various animations and works. The end goal is to ... Department of Psychology charan@ucdavis.edu Ranganath Lab Charan Ranganath\'s Dynamic Memory Lab @CharanRanganath Charan Ranganath is Director of the Memory and Plasticity Program and a Professor of Psychology and Neuroscience at the University of California at D

In [11]:
search = DuckDuckGoSearchResults()
search.run("Charan H U?")

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


'[snippet: Hello friends! Welcome to Charan H U YouTube channel. In this video I explained about Desk Setup Tour 2023! | ಡೆಸ್ಕ್ ಸೆಟಪ್ ಪ್ರವಾಸ 2023! | Charan H Uಚರಣ್ ..., title: Desk Setup Tour 2023! | ಡೆಸ್ಕ್ ಸೆಟಪ್ ಪ್ರವಾಸ 2023! | Charan H U, link: https://www.youtube.com/watch?v=1zoKLY2SEUE], [snippet: 1 Why explainability and transparency matter 2 What are the challenges of AI explainability and transparency 3 What are some strategies for AI explainability and transparency 4 What are some..., title: Strategies for AI Explainability and Transparency - LinkedIn, link: https://www.linkedin.com/advice/0/what-some-strategies-ai-explainability-transparency-wcp2c], [snippet: "We update our memories through the act of remembering," says Charan Ranganath, a professor of psychology and neuroscience at the University of California, Davis, and the author of the..., title: A Leading Memory Researcher Explains How to Make Precious Moments Last ..., link: https://www.nytimes.com/interactive/2024/02/04

In [12]:
search = DuckDuckGoSearchResults(backend="youtube")
search.run("Charan H U?")

''

In [19]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(time="d")

In [21]:
search = DuckDuckGoSearchRun()
search.run("'Who is Charan H U'?")

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


'Hello friends! Welcome to Charan H U YouTube channel. In this video I explained about Desk Setup Tour 2023! | ಡೆಸ್ಕ್ ಸೆಟಪ್ ಪ್ರವಾಸ 2023! | Charan H Uಚರಣ್ ... "We update our memories through the act of remembering," says Charan Ranganath, a professor of psychology and neuroscience at the University of California, Davis, and the author of the... Chiranjeevi. Konidela Chiranjeevi (born Konidela Sivasankara Varaprasad; 22 August 1955) is an Indian actor, film producer and former politician. He is regarded as one of the most successful and influential actors in the history of Indian cinema. [7] [8] In a career spanning over four decades, he starred in over 150 feature films predominantly ... Days after the government announced the prestigious Padma Vibhushan for Chiranjeevi, his son-actor Ram Charan and daughter-in-law Upasana hosted a dazzling celebration at the family\'s farmhouse on the outskirts of Hyderabad. Prominent personalities from the Telugu film industry and political arena atte